# Time-resolved emission case study of whole photosynthetic cells

Acuña AM, Van Alphen P, Van Grondelle R, Van Stokkum IHM (2018) The phycobilisome terminal emitter transfers its energy with a rate of (20 ps)–1 to photosystem II. Photosynthetica 56 (1):265-274. doi:10.1007/s11099-018-0779-7

# Inspect experimental data


In [ ]:
from pyglotaran_extras import plot_data_overview

experiment_data = {
    "DPSI400tr2": "experiment_data/RT400_590excDPSIjul30tr24KtargetPB10b.ascii",
    "DPSI590tr2": "experiment_data/RT400_590excDPSIjul30tr24KtargetPB10d.ascii",
    "DPSI400tr4": "experiment_data/RT400_590excDPSIjul30tr24KtargetPB10a.ascii",
    "DPSI590tr4": "experiment_data/RT400_590excDPSIjul30tr24KtargetPB10c.ascii",
    "data_guide_s1": "experiment_data/RT400_590excDPSIjul30tr24KtargetPB10e.ascii",
    "data_guide_s4": "experiment_data/RT400_590excDPSIjul30tr24KtargetPB10f.ascii",
}

plot_data_overview(experiment_data["DPSI400tr4"], linlog=True, linthresh=150);

# Create a project and import the data

In [ ]:
from glotaran.project import Project

project = Project.open("")
project.import_data(experiment_data)

# Model and Parameter definition


In [ ]:
# project.show_model_definition("model_DPSI_guidance_with_area_irf_fractions")

In [ ]:
# project.show_parameters_definition("optimized_parameters", as_dataframe=True)

# Optimization


some notes that can later be deleted
-  7         1.0352e+07 (no area penalty)
-  7         1.0355e+07 (with area penalty)
- 6              7         1.0352e+07      3.25e-02
- 3              4         1.0067e+07 (with 8 additional free scalem parameters)
-       6              7         1.0067e+07      5.57e-03       1.05e-04       1.25e+02    
`ftol` termination condition is satisfied.
- after changing (fractions) the megacomplex scale parameters 1.0068e+07
- note that the numbers differ from the published table, probably this is related to the normalization, but i am not sure.
- what probably is also different is the area penalty, here weight 0.01

In [ ]:
result = project.optimize(
    model_name="model_DPSI_guidance_with_area_irf_fractions",
    parameters_name="optimized_parameters",
    clp_link_tolerance=1.9,
    maximum_number_function_evaluations=7,
)

## Inspect fit quality


In [ ]:
result

In [ ]:
result.optimized_parameters

## Plot fitted traces


In [ ]:
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths

wavelengths = select_plot_wavelengths(result, axes_shape=[6, 5])
fig, axes = plot_fitted_traces(result, wavelengths, axes_shape=[6, 5], linlog=True, linthresh=150)
for ax in axes.flatten():
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title}$\,$nm")

In [ ]:
from pyglotaran_extras.plotting.plot_traces import plot_fitted_traces
from pyglotaran_extras.plotting.plot_traces import select_plot_wavelengths

wavelengths = [630, 650, 660, 680]
fig, axes = plot_fitted_traces(
    result,
    wavelengths,
    axes_shape=[2, 2],
    linlog=True,
    linthresh=100,
    figsize=(10, 7),
)
axes[1][0].set_xlabel("Time (ps)")
axes[1][1].set_xlabel("Time (ps)")
axes[0][0].set_xlabel("")
axes[0][0].set_ylabel("")
axes[1][0].set_ylabel("")
# axes[0][0].axhline(0, color="k", linewidth=1)
# axes[1][0].axhline(0, color="k", linewidth=1)
axes[0][1].set_xlabel("")
axes[0][1].set_ylabel("")
axes[1][1].set_ylabel("")
# axes[1][1].set_xlabel("Wavelength (nm)")
# axes[0][0].set_title("Concentrations")
# axes[0][1].set_title("SAS")
# axes[1][1].set_title("")
# axes[0][1].axhline(0, color="k", linewidth=1)
axes[0][0].annotate("A", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)
# axes[0][0].annotate("590 exc", xy=(0.01, 0.85), xycoords="axes fraction",fontsize=16)
# axes[1][0].annotate("400 exc", xy=(0.01, 0.85), xycoords="axes fraction",fontsize=16)
axes[0][1].annotate("B", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)
axes[1][0].annotate("C", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)
axes[1][1].annotate("D", xy=(0.01, 0.9), xycoords="axes fraction", fontsize=16)

for ax in axes.flatten():
    ax_title = ax.get_title()
    ax.set_title(rf"{ax_title}$\,$nm")

# Plot result for interpretation

## Overview


In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(result.data["DPSI590tr4"], linlog=True, linthresh=150, nr_of_residual_svd_vectors=1)

In [ ]:
plot_overview(result.data["DPSI400tr4"], linlog=True, linthresh=150, nr_of_residual_svd_vectors=1)

In [ ]:
combined_concentration = {
    "PC640": ["s8", "s9", "s10"],
    "PC650": ["s5", "s6", "s7"],
    "APC660": [
        "s1",
        "s2",
        "s3",
    ],
    "APC680": ["s4"],
    "PSII": ["s11", "s13"],
}

In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.style import ColorCode
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras.plotting.utils import add_cycler_if_not_none
from pyglotaran_extras.plotting.utils import extract_irf_location
from pyglotaran_extras.plotting.utils import shift_time_axis_by_irf_location

# myFRLcolors=['c','b','r','k','g','tab:grey','tab:orange',  'tab:purple','m','y', 'tab:brown']
myFRLcolors = [
    ColorCode.cyan,
    "b",
    "r",
    "k",
    ColorCode.green,
    "tab:grey",
    "tab:orange",
    "tab:purple",
    "m",
    "y",
    "tab:brown",
]
custom_cycler = cycler(color=myFRLcolors)

ds_name = "DPSI590tr4"


def only_existing_values(values, check_list):
    return [value for value in values if value in check_list]


concentration = result.data[ds_name].species_concentration
sas = result.data[ds_name].species_associated_spectra

concentration_dict = {}

for key, val in combined_concentration.items():
    species_labels = only_existing_values(combined_concentration[key], concentration.species)

    concentration_dict[key] = (
        concentration.sel(spectral=0, method="nearest")
        .sel(species=species_labels)
        .sum(dim="species")
        .drop("spectral")
    )


plot_dim = (2, 2)

# fig, axes = plt.subplots(*plot_dim, figsize=(10, 7),sharex='col')
fig, axes = plt.subplots(*plot_dim, figsize=(10, 7))
for ax in axes.flatten():
    add_cycler_if_not_none(ax, custom_cycler)
    # add_cycler_if_not_none(ax, PlotStyle().cycler)

irf_location = extract_irf_location(result.data[ds_name], 0)

for key, val in concentration_dict.items():
    shift_time_axis_by_irf_location(val, irf_location=irf_location).plot(
        x="time", label=key, ax=axes[0][0]
    )

for species in combined_concentration.values():
    species_labels = only_existing_values(species, sas.species)
    sas.sel(species=species_labels[0]).plot(x="spectral", label=species, ax=axes[0][1])


ds_name = "DPSI400tr4"


def only_existing_values(values, check_list):
    return [value for value in values if value in check_list]


concentration = result.data[ds_name].species_concentration
sas = result.data[ds_name].species_associated_spectra

concentration_dict = {}

for key, val in combined_concentration.items():
    species_labels = only_existing_values(combined_concentration[key], concentration.species)

    concentration_dict[key] = (
        concentration.sel(spectral=0, method="nearest")
        .sel(species=species_labels)
        .sum(dim="species")
        .drop("spectral")
    )

for ax in axes.flatten():
    add_cycler_if_not_none(ax, custom_cycler)
    # add_cycler_if_not_none(ax, PlotStyle().cycler)

irf_location = extract_irf_location(result.data[ds_name], 0)

for key, val in concentration_dict.items():
    shift_time_axis_by_irf_location(val, irf_location=irf_location).plot(
        x="time", label=key, ax=axes[1][0]
    )

for species in combined_concentration.values():
    species_labels = only_existing_values(species, sas.species)
    sas.sel(species=species_labels[0]).plot(x="spectral", label=species, ax=axes[1][1])

axes[0][0].set_xscale("symlog", linthresh=100, linscale=1)

axes[0][0].legend()

axes[1][0].set_xscale("symlog", linthresh=100, linscale=1)

# axes[1][0].legend()
axes[1][0].set_xlabel("Time (ps)")
axes[0][0].set_xlabel("")
axes[0][0].set_ylabel("")
axes[1][0].set_ylabel("")
axes[0][0].axhline(0, color="k", linewidth=1)
axes[1][0].axhline(0, color="k", linewidth=1)
axes[0][1].set_xlabel("")
axes[0][1].set_ylabel("")
axes[1][1].set_ylabel("")
axes[1][1].set_xlabel("Wavelength (nm)")
axes[0][0].set_title("Concentrations")
axes[0][1].set_title("SAS")
axes[1][1].set_title("")
axes[0][1].axhline(0, color="k", linewidth=1)
axes[1][1].axhline(0, color="k", linewidth=1)
axes[0][0].annotate("A", xy=(0.01, 0.93), xycoords="axes fraction", fontsize=16)
axes[0][0].annotate("590 exc", xy=(0.01, 0.85), xycoords="axes fraction", fontsize=16)
axes[1][0].annotate("400 exc", xy=(0.01, 0.85), xycoords="axes fraction", fontsize=16)
axes[0][1].annotate("B", xy=(0.01, 0.93), xycoords="axes fraction", fontsize=16)
axes[1][0].annotate("C", xy=(0.01, 0.93), xycoords="axes fraction", fontsize=16)
axes[1][1].annotate("D", xy=(0.01, 0.93), xycoords="axes fraction", fontsize=16)


fig.tight_layout()

## Fit of the guidance SAS

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.set_prop_cycle(PlotStyle().data_cycler_solid_dashed)
for data_set_suffix in ["s1", "s4"]:
    dataset_name = f"data_guide_{data_set_suffix}"
    result.data[dataset_name].data.plot(label=f"{dataset_name} data", ax=ax)
    result.data[dataset_name].fitted_data.plot(label=f"{dataset_name} fitted data", ax=ax)

ax.legend()
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("")
ax.set_title("Fit of the guidance SAS")
ax.axhline(0, color="k", linewidth=1)
# axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction",fontsize=16)

## Amplitude matrices of the target analysis for each dataset

In [ ]:
from pyglotaran_extras.inspect import show_a_matrixes

show_a_matrixes(result)

- NB1 with the non-transferring PB megacomplex (complex 5) there is no PSII compartment (s11 or s13) and the double sum equals 1 minus the PSII input at that wavelength.
- NB2 the dummy compartments s15 and s16 have zero SAS, and are used to accumulate the decay of the PB. thus with complex 1 (400 exc, open) we find a very small amount of emission lost via PB (7.7%), with complex 2 (400 exc, closed) it is 16.2%. with complex 3 (590 exc, open) we find 13.7% and with complex 4 (590 exc, closed) 21.6%. Thus one may conclude that in complexes of PB with open PSII RCs about 90% of the absorbed photons results in charge separation, and ~10% is lost via emission from PB.